# UCL Adult Dataset

The UCI Adult dataset, colloquially known as the "Census Income" dataset, represents extracted data from the 1994 Census bureau database. A classic dataset in the machine learning community, its main objective is to predict whether an individual earns more than $50,000 a year based on demographic variables.


### Dataset Overview:
- Age: Represents the age of an individual.
- Workclass: The general category of the individual's employment status (e.g., Private, Self-emp, Government).
- Education: The highest level of education attained by the individual.
- Marital-status: Marital status of the individual.
- Occupation: The general category of the individual's occupation (e.g., Tech-support, Craft-repair).
- Relationship: Represents what this individual is relative to others (e.g., Wife, Own-child, Husband).
- Race: Describes the individual's racial identity.
- Sex: The gender of the individual.
- Hours-per-week: Represents the number of hours the individual works per week.
- Native-country: The native country of the individual.
- Income: Binary outcome, indicating if the person makes more than 50K a year.


## Building a Data Pipeline

Before we embark on building our pipeline, let's first understand its significance. A data pipeline is a series of automated processes that allow data to flow seamlessly from its source to a destination, undergoing any necessary transformations along the way. This enables us to move data from diverse sources, refine it, and make it ready for analysis or other uses.



### Step 1: Import Dependencies

In [1]:
import pandas as pd
import sqlite3


### Step 2: Fetching the Data

In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", 
                "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"]

data = pd.read_csv(url, names=column_names, sep=r'\s*,\s*', engine='python')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Fetching data directly from the web allows us to always access the most up-to-date version of the dataset without the need for manual downloads.



### Step 3: Data Cleaning and Transformation


Raw data is often noisy, incomplete, and inconsistent. Cleaning ensures we're working with reliable data, while transforming it makes the data suit our specific analytical needs.

In [3]:
# Remove unnecessary columns
data = data.drop(columns=['fnlwgt', 'education-num', 'capital-gain', 'capital-loss'])

# Replace "?" with NaN for clarity
data.replace("?", pd.NA, inplace=True)

# Drop rows with missing values for consistency
data.dropna(inplace=True)

# Transform income column to simplify analysis
data['income'] = data['income'].apply(lambda x: 1 if x == ">50K" else 0)
data.head()

,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,0
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,0
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,0
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,0


This transformation simplifies the income column by converting it into a binary representation. Instead of strings ">50K" and "<=50K", we now have a binary variable where 1 represents high income (">50K") and 0 represents low income (<=50K). This makes the column much easier to use in analytical processes, especially in algorithms that require numerical input.


## Step 4: Storing the Data

Having a structured storage system like a database allows for efficient querying, updating, and ensures data integrity. It's a necessary step before any sophisticated analysis or machine learning.


In [4]:
# Connect to a SQLite database
conn = sqlite3.connect("adult_data.db")

# Store the dataframe into a new SQLite table
data.to_sql("adult_data", conn, if_exists="replace")

# Verify data integrity by fetching the first few rows
pd.read_sql_query("SELECT * FROM adult_data LIMIT 5", conn)

,index,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
0,0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,0
1,1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,0
2,2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,0
3,3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,0
4,4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,0


## Step 5: Building a Simple Querying Function

While storing data is crucial, being able to efficiently retrieve it for analysis is equally important. This function facilitates easy data querying.

In [5]:
def query_data(query, connection=conn):
    """
    Query the SQLite database and return results as a DataFrame.
    """
    return pd.read_sql_query(query, connection)

# Example usage:
query_data("SELECT age, occupation, income FROM adult_data WHERE age > 30 LIMIT 10")


,age,occupation,income
0,39,Adm-clerical,0
1,50,Exec-managerial,0
2,38,Handlers-cleaners,0
3,53,Handlers-cleaners,0
4,37,Exec-managerial,0
5,49,Other-service,0
6,52,Exec-managerial,1
7,31,Prof-specialty,1
8,42,Exec-managerial,1
9,37,Exec-managerial,1
